<a href="https://colab.research.google.com/github/zzhenxi/Competitions/blob/main/%5BDACON%5D%EB%89%B4%EC%8A%A4%EA%B7%B8%EB%A3%B9%EB%B6%84%EB%A5%98%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%ED%92%88%EC%82%AC%ED%83%9C%EA%B9%85%26TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 불러오기 & 전처리

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/dacon/train.csv')
test = pd.read_csv('/content/drive/MyDrive/dacon/test.csv')

In [ ]:
train.head()

,id,text,target
0,0,"\nThey were, and even if Washington might cons...",10
1,1,"We run ""SpaceNews & Views"" on our STAREACH BBS...",14
2,2,\n\n\nNot to worry. The Masons have been demo...,19
3,3,"Only Brendan McKay, or maybe ARF, would come t...",17
4,4,Help: I am running some sample problems from O...,5


In [ ]:
X = train.text
y = train.target

In [ ]:
import re

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 

        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\]\[\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
        
    return corpus

In [ ]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
print(train)

temp = clean_text(test['text']) #test셋에서도 똑같은 전처리 과정을 해줍니다.
test['text'] = temp

        id                                               text  target
0        0  they were and even if washington might conside...      10
1        1  we run spacenews views on our stareach bbs a l...      14
2        2  not to worry the masons have been demonized an...      19
3        3  only brendan mckay or maybe arf would come to ...      17
4        4  help i am running some sample problems from or...       5
...    ...                                                ...     ...
9228  9228  precisely why not cuba why not the hatians are...      17
9229  9229  your custom resume on disk macintosh or ibm co...       6
9230  9230  throughout the years of the israelarabpalestin...      17
9231  9231  does anyone know if there are any devices avai...       4
9232  9232  give me a break chum are you telling me that c...      18

[9233 rows x 3 columns]


In [ ]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의합니다.

In [ ]:
import nltk
from nltk.corpus import stopwords
from os import path
nltk.download('all')

In [ ]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


In [ ]:
from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


In [ ]:
# pos_tag()의 입력값으로는 단어의 리스트가 들어가야 한다.
print(nltk.pos_tag(nltk.word_tokenize(train['text'][4])))

[('help', 'NN'), ('i', 'VB'), ('am', 'VBP'), ('running', 'VBG'), ('some', 'DT'), ('sample', 'NN'), ('problems', 'NNS'), ('from', 'IN'), ('oreilly', 'RB'), ('volume', 'NN'), ('xt', 'JJ'), ('intrisics', 'NNS'), ('programming', 'VBG'), ('manual', 'JJ'), ('chapter', 'NN'), ('popupdialog', 'NN'), ('boxes', 'NNS'), ('and', 'CC'), ('so', 'RB'), ('onin', 'JJ'), ('example', 'NN'), ('page', 'NN'), ('creating', 'VBG'), ('a', 'DT'), ('popup', 'NN'), ('dialog', 'NN'), ('boxthe', 'JJ'), ('application', 'NN'), ('creates', 'VBZ'), ('window', 'VBP'), ('with', 'IN'), ('a', 'DT'), ('button', 'NN'), ('quit', 'NN'), ('and', 'CC'), ('press', 'NN'), ('methe', 'VBP'), ('button', 'NN'), ('press', 'NN'), ('me', 'PRP'), ('pops', 'VBZ'), ('up', 'RP'), ('a', 'DT'), ('dialog', 'NN'), ('box', 'IN'), ('the', 'DT'), ('strange', 'JJ'), ('feature', 'NN'), ('ofthis', 'JJ'), ('program', 'NN'), ('is', 'VBZ'), ('that', 'IN'), ('it', 'PRP'), ('always', 'RB'), ('pops', 'VBZ'), ('up', 'RP'), ('the', 'DT'), ('dialog', 'NN'), ('

명사, 동사, 형용사, 부사를 추출하여 사용

In [ ]:
def postagging(dataframe):
    main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
    for sentence in dataframe['text']: # 리뷰들을 하나씩 가져옵니다
        pos = nltk.pos_tag(nltk.word_tokenize(sentence)) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
        main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ('JJ', 'JJR', 'JJS', #형용사
                                                                         'NN', 'NNS', 'NNP', 'NNPS', #명사
                                                                         'RB', 'RBR', 'RBBS', #부사
                                                                         'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ' #동사
                                                                         )] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
        main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
        main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
        dataframe["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

postagging(train)
postagging(test) ## test셋도 똑같이 품사태깅을 적용해줍니다.
train.head() # 데이터 확인

,id,text,target,main_pos
0,0,they were and even if washington might conside...,10,were even washington consider patty bust id tr...
1,1,we run spacenews views on our stareach bbs a l...,14,run spacenews views stareach bbs localoperatio...
2,2,not to worry the masons have been demonized an...,19,not worry masons have been demonized harrassed...
3,3,only brendan mckay or maybe arf would come to ...,17,only brendan mckay maybe arf come rescue nazir...
4,4,help i am running some sample problems from or...,5,help i am running sample problems oreilly volu...


In [ ]:
from sklearn.model_selection import train_test_split

data = train.copy()
train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [ ]:
print( 'train 데이터 셋 모양 :', train.shape)
print( 'val 데이터 셋 모양 :', val.shape)

train 데이터 셋 모양 : (6924, 5)
val 데이터 셋 모양 : (2309, 5)


In [ ]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

In [ ]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression(max_iter=500) #객체에 모델 할당
model.fit(X_train_vec, y_train) #모델 학습

LogisticRegression(max_iter=500)

In [ ]:
# 검증
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

In [ ]:
y_pred = model.predict(X_val_vec)
print(y_pred)

[ 7  9  6 ...  5 12  6]


In [ ]:
from sklearn import metrics
print('Logistic Regression 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred),3)) # 정확도 확인

Logistic Regression 의 예측 정확도는 0.593


tf-idf사용하기

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_val_tfidf_vect = tfidf_vect.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform
print('학습 & 테스트 데이터 Text의 TfidfVectorizer Shape:',X_train_tfidf_vect.shape, X_val_tfidf_vect.shape)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_val_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(metrics.accuracy_score(y_val ,pred)))

학습 & 테스트 데이터 Text의 TfidfVectorizer Shape: (6924, 639663) (2309, 639663)
TF-IDF Logistic Regression 의 예측 정확도는 0.717


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.target #training 데이터에서 라벨 추출

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.target #validation 데이터에서 라벨 추출

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_val_tfidf_vect = tfidf_vect.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform
print('학습 & 테스트 데이터 Text의 TfidfVectorizer Shape:',X_train_tfidf_vect.shape, X_val_tfidf_vect.shape)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_val_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(metrics.accuracy_score(y_val ,pred)))

학습 & 테스트 데이터 Text의 TfidfVectorizer Shape: (6924, 639663) (2309, 639663)
TF-IDF Logistic Regression 의 예측 정확도는 0.717


In [ ]:
X_train = data.main_pos #전체 training 데이터에서 문서 추출
y_train = data.target #전체 training 데이터에서 라벨 추출

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)

LogisticRegression(C=10, solver='liblinear')

In [ ]:
X_test = test.main_pos
X_test_vec = tfidf_vect.transform(X_test)
pred_test = lr_clf.predict(X_test_vec)
print(pred_test)

[ 3 16 11 ...  4  1 12]


In [ ]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('/content/drive/MyDrive/dacon/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['target'] = pred_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,target
0,0,3
1,1,16
2,2,11
3,3,8
4,4,13
...,...,...
9228,9228,16
9229,9229,1
9230,9230,4
9231,9231,1


In [ ]:
submission.to_csv("submission_baseline2_zhenxi.csv", index=False)

In [ ]:
pwd

'/content'